# converting signals into images, bin files(.npy)

いくつかのファイルが内部にありますが、このノートブックではディレクトリを指定して、その中にある生波形データ(.mat)を、一括して画像(.png)及びその値（.npy）に変換し保存するという処理を行っています。

In [ ]:
import os
import sys
from src import generate_echomap
from datetime import datetime

# 入力ディレクトリと出力ディレクトリの設定
input_dir = r"Z:\database\signal_mat"
output_dir = r"Z:\database\images\detail_img"
log_file = os.path.join(output_dir, f"processing_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")

# 出力ディレクトリが存在しない場合は作成
os.makedirs(output_dir, exist_ok=True)

# 標準出力をファイルにリダイレクト
original_stdout = sys.stdout
with open(log_file, 'w', encoding='utf-8') as f:
    sys.stdout = f
    
    print(f"処理開始時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"入力ディレクトリ: {input_dir}")
    print(f"出力ディレクトリ: {output_dir}")
    print("-" * 50)
    
    # 入力ディレクトリ内のすべての.matファイルを処理
    for filename in os.listdir(input_dir):
        if filename.endswith('.mat'):
            file_path = os.path.join(input_dir, filename)
            print(f"処理中: {filename}")
            
            try:
                # generate_echomapを実行
                triggered_pulses, adjusted_time_us, mean_pulse, std_pulse = generate_echomap(
                    file_path, 
                    start_time=2, 
                    duration=0.02,
                    window_width=100e-6, 
                    amplitude_threshold=2.0, 
                    output_dir=output_dir)
                
                print(f"完了: {filename}")
            except Exception as e:
                print(f"エラー発生 ({filename}): {str(e)}")
    
    print("-" * 50)
    print(f"すべてのファイル処理が完了しました。")
    print(f"処理終了時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# 標準出力を元に戻す
sys.stdout = original_stdout
print(f"処理が完了しました。ログは {log_file} に保存されています。")

In [ ]:
import os
import glob
from src import generate_bin_multi
# 処理対象のディレクトリ
input_dir = "Z:\database\samples"
output_dir = "D:\csvfiles\sandbox"

# .matファイルのリストを取得
mat_files = glob.glob(os.path.join(input_dir, "*.mat"))

# 処理するチャンネル
channels = ["TDX1", "TDX2", "TDX3", "TDX1_enlarged"]

# 各ファイルに対して処理を実行
for mat_file in mat_files:
    print(f"Processing file: {mat_file}")
    try:
        generate_bin_multi(
            mat_file,
            channels=channels,
            start_time=0.0,
            duration=5,
            starting_window=55e-6,
            ending_window=105e-6,
            amplitude_threshold=2.0,
            output_dir=output_dir
        )
        print(f"Successfully processed: {mat_file}")
    except Exception as e:
        print(f"Error processing {mat_file}: {str(e)}")